In [109]:
import pickle
import numpy as np
import re
from utils import clean, split_on_caps, TOP_100_PLAYERS
import pandas as pd
from nltk import sent_tokenize #### rememeber this
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import nltk
from nltk.corpus import stopwords
from gensim.models.ldamodel import LdaModel
from gensim import corpora

STOPLIST = stopwords.words('english')

import codecs
import json

## Load Data

In [113]:
docs_cleaned_filename = "docs_cleaned_saturday"
with open (docs_cleaned_filename, 'rb') as fp:
    docs = pickle.load(fp)
    
article_dicts_filename = "article_dicts_saturday"
all_articles = []
with codecs.open(article_dicts_filename,'rU','utf-8') as f:
    for line in f:
        all_articles.append(json.loads(line))

In [114]:
article_dicts_filename = "article_dicts_saturday"
all_articles = []
with codecs.open(article_dicts_filename,'rU','utf-8') as f:
    for line in f:
        all_articles.append(json.loads(line))

In [116]:
len(docs), len(all_articles)

(3314, 3314)

### LDA on all the documents

In [115]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    ps = PosterStemmer()
    return [ps.stem(item) for item in tokens]

#### Use TF-IDF to identify screen most important words

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english', min_df = 0.01, max_df = 0.35)

trans = tfidf.fit_transform(docs)
feats = tfidf.get_feature_names()
vocab = set(tfidf.vocabulary_.keys())

In [23]:
texts = [[word for word in tokenize(text) if word in vocab and word.isalpha()] for text in docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [84]:
%%time
NUM_TOPICS = 30
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=NUM_TOPICS, update_every=1, chunksize=5000, passes=4)

CPU times: user 25.5 s, sys: 446 ms, total: 26 s
Wall time: 26.3 s


In [117]:
# for i in range(NUM_TOPICS):
#     for word_id, p in lda.get_topic_terms(i):
#         print(dictionary[word_id],p, end=" ")
#     print("\n")

In [81]:
# for word_id, prob in lda.get_topic_terms(3)[:5]:
#     print(dictionary[word_id], prob, end=" ")
# print("\n")

million 0.024970928 trade 0.02018343 pick 0.016988372 free 0.014145047 deal 0.011659104 



In [94]:
# from collections import defaultdict
# topic2docs = defaultdict(list)
# for i,corpus_doc in enumerate(corpus):
#     topic_id = max(lda[corpus_doc], key=lambda x:x[1])[0]
#     topic2docs[topic_id].append(i)